<a href="https://colab.research.google.com/github/swanilee/-8-8/blob/master/AvatarCLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AvatarCLIP: Zero-Shot Text-Driven Generation and Animation of 3D Avatars

Authored by Fangzhou Hong, Mingyuan Zhang, Liang Pan, Zhongang Cai, Lei Yang, Ziwei Liu

This colab notebook provides a quick example of avatar shape sculpting and texture generation part described in our paper.

For the coarse shape generation and motion generation, please refer to our official Github repo (https://github.com/hongfz16/AvatarCLIP).

Please note that due to the GPU memory limitation, this demo cannot fully achieve the quality shown in the paper. To fit in the smaller GPU memory, we scale down the model, which harms its capacity. We also tune down the rendering resolution to reduce memory usage which will hurt the supervision from CLIP.

For more information, feel free to visit our project page (https://hongfz16.github.io/projects/AvatarCLIP.html).

The whole optimization process might take 4-10 hours depending on the machine you get allocated. Empirically, you can check the results after 10,000 iterations.

In [1]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

## Setup SMPL Models

Please register and download SMPL models here(https://smpl.is.tue.mpg.de/). Create folders under AvatarCLIP and put SMPL_NEUTRAL.pkl under the folder as follows.

```
./AvatarCLIP/
├── ...
└── smpl_models/
    └── smpl/
        └── SMPL_NEUTRAL.pkl
```

In [2]:
# @title Setup the Environment: 1. Common Dependencies
!nvidia-smi
!git clone https://github.com/hongfz16/AvatarCLIP.git
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r AvatarCLIP/requirements.txt
!git clone https://github.com/hongfz16/neural_renderer.git

Tue Aug  8 08:23:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# @title Setup the Environment: 2. Modify neural_renderer
with open('neural_renderer/neural_renderer/perspective.py', 'r') as f:
  lines = f.readlines()
lines.insert(19, '    x[z<=0] = 0\n')
lines.insert(20, '    y[z<=0] = 0\n')
lines.insert(21, '    z[z<=0] = 0\n')
with open('neural_renderer/neural_renderer/perspective.py', 'w') as f:
  for l in lines:
    f.write(l)

In [4]:
# @title Setup the Environment: 3. Compile neural_renderer
%cd /content
%cd neural_renderer
!python3 setup.py install
%cd ../AvatarCLIP
%cd AvatarGen/AppearanceGen

/content
/content/neural_renderer
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build 

In [5]:
#####################################################################
# This code block is for myself to quickly setup SMPL models.       #
# Due to the license restriction, I cannot provide the SMPL models. #
# Please register and download at https://smpl.is.tue.mpg.de/       #
#####################################################################

%cd /content/AvatarCLIP/AvatarGen/AppearanceGen
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir -p ../../smpl_models/smpl
!cp '/content/smpl_models/smpl/SMPL_NEUTRAL.pkl' ../../smpl_models/smpl/SMPL_NEUTRAL.pkl

/content/AvatarCLIP/AvatarGen/AppearanceGen
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Important: Restart Runtime Before Proceeding (Menu => Runtime => Restart and run all)

This is for the self compiled module (neural_renderer) to be activated.

In [6]:
!pip install trimesh
!pip install icecream
!pip install pyhocon

In [10]:
#@title Import Necessary Modules
%cd /content/AvatarCLIP/AvatarGen/AppearanceGen
! pip install PyMCubes
! pip install smplx
! pip install git+https://github.com/openai/CLIP.git #pip instasll clip
! pip install pyhocon==0.3.43
! pip install chumpy
import os
import time
import logging
import argparse
import random
import numpy as np
import cv2 as cv
import trimesh
import torch
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from shutil import copyfile
from icecream import ic
from tqdm import tqdm
from pyhocon import ConfigFactory
from models.dataset import Dataset
from models.dataset import SMPL_Dataset
from models.fields import RenderingNetwork, SDFNetwork, SingleVarianceNetwork, NeRF
from models.renderer import NeuSRenderer
from models.utils import lookat, random_eye, random_at, render_one_batch, batch_rodrigues
from models.utils import sphere_coord, random_eye_normal, rgb2hsv, differentiable_histogram
from models.utils import my_lbs, readOBJ
import clip
from smplx import build_layer
import imageio

to8b = lambda x : (255*np.clip(x,0,1)).astype(np.uint8)
from main import Runner

/content/AvatarCLIP/AvatarGen/AppearanceGen
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-dlxl8_aw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-dlxl8_aw
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58266 sha256=2d96c531be88809fc9c4ef1fd9def95d297ec222e9351458a1eadf18e7ac41f2
  Stored in directory: /root/.cache/pip/wheels/e0/c1/ef/29ba7be03653a29ef6f2c3e1956d6c4d8877f2b243af411db1
Successfully built chumpy


In [11]:
#@title Input Appearance Description (e.g. Iron Man)
AppearanceDescription = "Iron Man" #@param {type:"string"}

torch.set_default_tensor_type('torch.cuda.FloatTensor')
FORMAT = "[%(filename)s:%(lineno)s - %(funcName)20s() ] %(message)s"
logging.basicConfig(level=logging.INFO, format=FORMAT)
conf_path = 'confs/examples_small/example.conf'
f = open(conf_path)
conf_text = f.read()
f.close()
conf_text = conf_text.replace('{TOREPLACE}', AppearanceDescription)
# print(conf_text)
!pip install pyhocon==0.3.43
from pyhocon import ConfigFactory
conf = ConfigFactory.parse_string(conf_text)
print("Prompt: {}".format(conf.get_string('clip.prompt')))
print("Face Prompt: {}".format(conf.get_string('clip.face_prompt')))
print("Back Prompt: {}".format(conf.get_string('clip.back_prompt')))

# clip_prompt = "a 3D rendering of a Iron Man in unreal engine"
# clip_face_prompt = "a 3D rendering of the face of Iron Man in unreal engine"
# clip_back_prompt = "a 3D rendering of the back of Iron Man in unreal engine"

# print("Prompt: {}".format(clip_prompt))
# print("Face Prompt: {}".format(clip_face_prompt))
# print("Back Prompt: {}".format(clip_back_prompt))


Prompt: a 3D rendering of a Iron Man in unreal engine
Face Prompt: a 3D rendering of the face of Iron Man in unreal engine
Back Prompt: a 3D rendering of the back of Iron Man in unreal engine


In [ ]:
#@title Start the Generation!
# !pip3 install clip-by-openai
runner = Runner(conf_path, 'train_clip', 'smpl', False, True, conf)
runner.init_clip()
runner.init_smpl()
runner.train_clip()

Load data: Begin
Load data: End
Use head height: 0.7
Prompt: a 3D rendering of a Iron Man in unreal engine
Face Prompt: a 3D rendering of the face of Iron Man in unreal engine
Back Prompt: a 3D rendering of the back of Iron Man in unreal engine


  0%|          | 0/30000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: upda

./exp/smpl/example
iter:100 loss = 1.5702341794967651 lr=9.900000000000001e-05
Validate: iter: 100, camera: 58


  1%|          | 199/30000 [01:59<4:27:41,  1.86it/s]

./exp/smpl/example
iter:200 loss = 1.5722421407699585 lr=0.000199
Validate: iter: 200, camera: 58


...

 85%|████████▌ | 25599/30000 [4:19:10<40:40,  1.80it/s]

./exp/smpl/example
iter:25600 loss = 1.6344373226165771 lr=5.061106085696087e-05
Validate: iter: 25600, camera: 58


 86%|████████▌ | 25699/30000 [4:20:08<39:00,  1.84it/s]

./exp/smpl/example
iter:25700 loss = 1.5168228149414062 lr=4.948060650936371e-05
Validate: iter: 25700, camera: 58


 86%|████████▌ | 25708/30000 [4:20:15<43:28,  1.65it/s]

In [ ]:
#@title Generate a Video of Optimization Process (RGB)

import os
from tqdm import tqdm
import numpy as np
from IPython import display
from PIL import Image
from base64 import b64encode

image_folder = 'exp/smpl/example/validations_extra_fine'
image_fname = os.listdir(image_folder)
image_fname = sorted(image_fname)
image_fname = [os.path.join(image_folder, f) for f in image_fname]

init_frame = 1
last_frame = len(image_fname)
min_fps = 10
max_fps = 60
total_frames = last_frame - init_frame

length = 15 #Desired time of the video in seconds

frames = []
for i in range(init_frame, last_frame): #
    frames.append(Image.open(image_fname[i]))

#fps = last_frame/10
fps = np.clip(total_frames/length,min_fps,max_fps)

from subprocess import Popen, PIPE
p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video.mp4'], stdin=PIPE)
for im in tqdm(frames):
    im.save(p.stdin, 'PNG')
p.stdin.close()
p.wait()
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

display.HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#@title Generate a Video of Optimization Process (Normal)

import os
from tqdm import tqdm
import numpy as np
from IPython import display
from PIL import Image
from base64 import b64encode

image_folder = 'exp/smpl/example/normals'
image_fname = os.listdir(image_folder)
image_fname = sorted(image_fname)
image_fname = [os.path.join(image_folder, f) for f in image_fname]

init_frame = 1
last_frame = len(image_fname)
min_fps = 10
max_fps = 60
total_frames = last_frame - init_frame

length = 15 #Desired time of the video in seconds

frames = []
for i in range(init_frame, last_frame): #
    frames.append(Image.open(image_fname[i]))

#fps = last_frame/10
fps = np.clip(total_frames/length,min_fps,max_fps)

from subprocess import Popen, PIPE
p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video_normal.mp4'], stdin=PIPE)
for im in tqdm(frames):
    im.save(p.stdin, 'PNG')
p.stdin.close()
p.wait()
mp4 = open('video_normal.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

display.HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

#### Finally, don't forget to download the extracted meshes at `AvatarCLIP/AvatarGen/AppearanceGen/exp/smpl/example/meshes/*.ply`